# Twitter Sentiment Analysis using Twitter **API**

In [23]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [24]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install textblob


Note: you may need to restart the kernel to use updated packages.


In [26]:
import numpy as np
import pandas as pd
import tweepy
from tweepy import OAuthHandler
import json


In [27]:
consumer_key = "SXM2tDHfi54Gpj2kiyvfwa8ij"
consumer_secret_key = "HEv12Up8P49105aOwqMntn96KuQTOOvpTe0mYGvQVVlNzlJBYL"
access_token = "32469346-pWnNjXWkZ6RzuNHNstLICRmmYvE8nz40wlWdPyTbA"
access_secret_token = "w77Fa3gUGC7r828aaMnHGxUwVytNbeNCMnvfnj0"


**Creating the API Connect to Pull Data**

In [28]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_token, access_secret_token)
api = tweepy.API(auth)

## Passing the **Query and Formulating all Tweets **to display:

In [29]:
query = "Covid-19 cases in USA"

In [30]:
Tweets = api.search(query, count=50, lang='en', exclude='retweets', tweet_mode='extended')

for tweet in Tweets:
    print(tweet,"\n")
    print(".............................\n")

Status(_api=<tweepy.api.API object at 0x1a244a0d50>, _json={'created_at': 'Thu Apr 01 06:42:29 +0000 2021', 'id': 1377511671003025409, 'id_str': '1377511671003025409', 'full_text': 'Nations challenge WHO report on coronavirus origins; US adds 4,300 variant cases in last week: Latest COVID-19 updates - USA TODAY\nhttps://t.co/xcvsY11IcK', 'truncated': False, 'display_text_range': [0, 154], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [], 'urls': [{'url': 'https://t.co/xcvsY11IcK', 'expanded_url': 'https://newsfeeds.media/nations-challenge-who-report-on-coronavirus-origins-us-adds-4300-variant-cases-in-last-week-latest-covid-19-updates-usa-today/', 'display_url': 'newsfeeds.media/nations-challe…', 'indices': [131, 154]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://newsfeeds.media" rel="nofollow">Newsfeedsmedia</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_t

# **Creating a Data Frame for Tweets pulled from Twitter**

In [31]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [32]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [33]:
stream(data=[" Covid-19 vaccination"] , file_name='my_tweets')

In [34]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date
0,@_PeppaJack Tip for a #Panther fan\n\nhttps://...,David Lanning,22962,96,"North Carolina, USA",False,0,0,2021-04-01 11:46:10
1,RT @OpIndia_com: Muslims reluctant to get vacc...,Shraddha Shah,2996,152,India 🇮🇳,False,0,220,2021-04-01 11:46:09
2,"Countries such as the United Kingdom, Israel a...",Global Edmonton,199164,222696,"Edmonton, AB",True,0,0,2021-04-01 11:46:05
3,RT @WRALSarah: This is what some in #Durham ha...,"Rebecca Feinglos Planchard, MPP",1469,655,"Durham, NC",False,0,5,2021-04-01 11:45:48
4,RT @wef: 5 questions you might have on COVID-1...,Zoryna O'Donnell,57624,3417,"Isle of Wight, United Kingdom",False,0,52,2021-04-01 11:45:45


**Sentiment**(polarity, subjectivity)

**Polarity score**: [-1 , +1]

**Subjectivity**: [0.0 , 1.0] where 0.0 is very Objective and 1.0 is very Subjective.

In [35]:
from textblob import TextBlob

In [36]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [37]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return "Positive Sentiment"
    elif analysis.sentiment.polarity == 0:
        return "Neutral Sentiment"
    else:
        return "Negative Sentiment"

In [38]:
df['clean_tweet']= df['Tweets'].apply(lambda x: clean_tweet(x))

In [39]:
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet
0,@_PeppaJack Tip for a #Panther fan\n\nhttps://...,David Lanning,22962,96,"North Carolina, USA",False,0,0,2021-04-01 11:46:10,PeppaJack Tip for a Panther fan
1,RT @OpIndia_com: Muslims reluctant to get vacc...,Shraddha Shah,2996,152,India 🇮🇳,False,0,220,2021-04-01 11:46:09,RT com Muslims reluctant to get vaccinated var...
2,"Countries such as the United Kingdom, Israel a...",Global Edmonton,199164,222696,"Edmonton, AB",True,0,0,2021-04-01 11:46:05,Countries such as the United Kingdom Israel an...
3,RT @WRALSarah: This is what some in #Durham ha...,"Rebecca Feinglos Planchard, MPP",1469,655,"Durham, NC",False,0,5,2021-04-01 11:45:48,RT This is what some in Durham have been calli...
4,RT @wef: 5 questions you might have on COVID-1...,Zoryna O'Donnell,57624,3417,"Isle of Wight, United Kingdom",False,0,52,2021-04-01 11:45:45,RT 5 questions you might have on COVID 19 vacc...


In [40]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

,Tweets,User,User_statuses_count,user_followers,User_location,User_verified,fav_count,rt_count,tweet_date,clean_tweet,Sentiment
0,@_PeppaJack Tip for a #Panther fan\n\nhttps://...,David Lanning,22962,96,"North Carolina, USA",False,0,0,2021-04-01 11:46:10,PeppaJack Tip for a Panther fan,Neutral Sentiment
1,RT @OpIndia_com: Muslims reluctant to get vacc...,Shraddha Shah,2996,152,India 🇮🇳,False,0,220,2021-04-01 11:46:09,RT com Muslims reluctant to get vaccinated var...,Negative Sentiment
2,"Countries such as the United Kingdom, Israel a...",Global Edmonton,199164,222696,"Edmonton, AB",True,0,0,2021-04-01 11:46:05,Countries such as the United Kingdom Israel an...,Positive Sentiment
3,RT @WRALSarah: This is what some in #Durham ha...,"Rebecca Feinglos Planchard, MPP",1469,655,"Durham, NC",False,0,5,2021-04-01 11:45:48,RT This is what some in Durham have been calli...,Positive Sentiment
4,RT @wef: 5 questions you might have on COVID-1...,Zoryna O'Donnell,57624,3417,"Isle of Wight, United Kingdom",False,0,52,2021-04-01 11:45:45,RT 5 questions you might have on COVID 19 vacc...,Neutral Sentiment


In [50]:
n = 65
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @ddnewsSrinagar: Covid-19 Vaccination drive for 45 years and above age category kick started today at Court Complex #Budgam. https://t.c…

Clean tweet:
 RT Covid 19 Vaccination drive for 45 years and above age category kick started today at Court Complex Budgam

Sentiment of the tweet:
 Negative Sentiment


In [45]:
n = 50
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @JamaicaObserver: TOURISM Minister Edmund Bartlett has cautioned global leaders that any requirement for proof of vaccination for travel…

Clean tweet:
 RT TOURISM Minister Edmund Bartlett has cautioned global leaders that any requirement for proof of vaccination for travel

Sentiment of the tweet:
 Neutral Sentiment


In [46]:
n = 30
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @NHSEngland: 'COVID-19 has kept us apart for far too long. We want to hug you, we want to celebrate with you, we want to go out for dinn…

Clean tweet:
 RT COVID 19 has kept us apart for far too long We want to hug you we want to celebrate with you we want to go out for dinn

Sentiment of the tweet:
 Positive Sentiment


In [42]:
df[df.Sentiment == 'Positive Sentiment'].shape[0]

406

In [43]:
df[df.Sentiment == 'Neutral Sentiment'].shape[0]

430

In [44]:
df[df.Sentiment == 'Negative Sentiment'].shape[0]

164